In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

C:\Users\amukh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = torch.tensor(pd.read_csv("mnist_train.csv", header=None).values)
x, y = (data[:, 1:]/255).float(), torch.nn.functional.one_hot(data[:, 0].long(), 10)

test = torch.tensor(pd.read_csv("mnist_test.csv", header=None).values)
x_test, y_test = (test[:, 1:]/255).float(), torch.nn.functional.one_hot(test[:, 0].long(), 10)

In [3]:
class Real(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = Real()
optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
losses = []

In [4]:
for epoch in tqdm(range(100)):
	for i in range(0, len(x), 100):
		batch_x, batch_y = x[i:i+100], y[i:i+100].float()
		optimiser.zero_grad()
		output = model(batch_x)
		loss = F.mse_loss(output, batch_y)
		loss.backward()
		optimiser.step()
	losses.append(loss.item())

 99%|█████████▉| 99/100 [03:17<00:02,  2.15s/it]

In [ ]:
accuracy = (model(x).argmax(1) == y.argmax(1)).sum().item()/len(x)
print(f"Training Accuracy: {accuracy*100:.2f}%")
accuracy = (model(x_test).argmax(1) == y_test.argmax(1)).sum().item()/len(x_test)
print(f"Testing Accuracy: {accuracy*100:.2f}%")
plt.plot(range(len(losses)), losses)